In [1]:
import pandas as pd
import requests

In [2]:
base_event_url = 'https://esports-api.lolesports.com/persisted/gw/getCompletedEvents?hl=en-GB&tournamentId={tournament_id}'
auth_token = '0TvQnueqKa5mxJntVWt0w4LpLfEkrV1Ta8rQBb9Z'

In [25]:
tournament_df = pd.read_csv('../data/tournament_codes.csv')

In [26]:
tournament_df= tournament_df[(tournament_df['league'].isin(['lec','lcs','lcl','lck'])) & (tournament_df['year'] == '2024' ) & (tournament_df['season'] != 'challengers' )]

In [27]:
tournament_df

,league,id,slug,startDate,endDate,year,season
0,lec,112352881163915249,lec_summer_2024,2024-06-08,2024-07-28,2024,summer
1,lec,111997906550466231,lec_spring_2024,2024-03-08,2024-04-15,2024,spring
2,lec,111560983131400452,lec_winter_2024,2024-01-12,2024-02-19,2024,winter
27,lcs,111504625283627681,lcs_spring_2024,2024-01-19,2024-04-01,2024,spring
74,lck,112435564213994193,lck_summer_2024,2024-06-12,2024-09-08,2024,summer
75,lck,111561337005798024,lck_spring_2024,2024-01-16,2024-04-15,2024,spring


In [77]:
def process_match_data(json_data):
    new_record = dict()
    new_record['startTime'] = json_data['startTime']
    new_record['blockName'] = json_data['blockName']
    new_record['league'] = json_data['league']['name']
    new_record['match_id'] = json_data['match']['id']
    new_record['type'] = json_data['match']['type']
    new_record['team_1_name'] = json_data['match']['teams'][0]['name']
    new_record['team_1_code'] = json_data['match']['teams'][0]['code']
    new_record['team_1_result'] = json_data['match']['teams'][0]['result']['gameWins']
    new_record['team_2_name'] = json_data['match']['teams'][1]['name']
    new_record['team_2_code'] = json_data['match']['teams'][1]['code']
    new_record['team_2_result'] = json_data['match']['teams'][1]['result']['gameWins']

    new_record['format'] = json_data['match']['strategy']['type'] +'_'+ str(json_data['match']['strategy']['count'])
    games_list = []
    for g in json_data['games']:
        games_list.append([g['id'],g['vods']])

    new_record['games'] = games_list

    return new_record

In [81]:
temp_list = []
for league in tournament_df['id']:
    url = base_event_url.format(tournament_id = league)
    json_data = requests.get(url,headers={'x-api-key': auth_token}).json()
    if len(json_data['data']['schedule']['events']) == 0:
        continue
    
    for event in json_data['data']['schedule']['events']:
        new_record = process_match_data(event)
        temp_list.append(new_record)

In [83]:

event_df = pd.DataFrame.from_dict(temp_list)

In [86]:
event_df.to_csv('../data/event.csv',index=False)